In [2]:
# Tutorial courtesy of https://stackabuse.com/image-recognition-in-python-with-tensorflow-and-keras/

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from tensorflow.python.keras.layers.convolutional import Conv2D, MaxPooling2D 
from tensorflow.python.keras.constraints import maxnorm
from tensorflow.python.keras.utils import np_utils

from tensorflow.keras.datasets import cifar10

## Prep data

In [ ]:
#Load data
(xtrain, ytrain), (xtest, ytest) = cifar10.load_data()

In [15]:
#Normalize the inputs from 0-255 to between 0 and 1 by dividing by 255
xtrain = xtrain.astype('float32')
xtest = xtest.astype('float32')
xtrain = xtrain / 255.0
xtest = xtest / 255.0

In [17]:
# one hot encode outputs
ytrain = np_utils.to_categorical(ytrain)
ytest = np_utils.to_categorical(ytest)
classNum = ytest.shape[1]

## Create model

In [24]:
model = Sequential()

#Convolutional layer runs convolutional filters on inputs
#32 filters of size 3x3, specifies shape for first layer
model.add(Conv2D(32, (3, 3), input_shape=xtrain.shape[1:], padding='same'))
model.add(Activation('relu'))

#Dropout layer prevents overfitting
#Drops 20% of existing connections
model.add(Dropout(0.2))

#Batch normalization normalizes input
model.add(BatchNormalization())

#Another convolutional layer with larger filter size
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))

#Pooling layer compresses image
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

#Repeat layers again
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

In [25]:
#Flatten data into one vector
model.add(Flatten())
model.add(Dropout(0.2))

#Dense layers form collections of neurons that form objects
model.add(Dense(256, kernel_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
    
model.add(Dense(128, kernel_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

#Final layer outputs a 10 neuron vector storing probability of each class
#Softmax activation selects neuron with highest probability as output
model.add(Dense(classNum))
model.add(Activation('softmax'))

## Compile model

In [29]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics= ['accuracy'])


## Train model

In [27]:
seed = 21
np.random.seed(seed)
with tf.device('/device:GPU:0'):
    model.fit(xtrain, ytrain, validation_data=(xtest,ytest), epochs=10, batch_size=32)


Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 137s 3ms/sample - loss: 1.4232 - acc: 0.4966 - val_loss: 1.1434 - val_acc: 0.5800
Epoch 2/10
50000/50000 [==============================] - 137s 3ms/sample - loss: 1.0237 - acc: 0.6378 - val_loss: 0.9456 - val_acc: 0.6687
Epoch 3/10
50000/50000 [==============================] - 138s 3ms/sample - loss: 0.9346 - acc: 0.6730 - val_loss: 0.9203 - val_acc: 0.6686
Epoch 4/10
50000/50000 [==============================] - 138s 3ms/sample - loss: 0.8942 - acc: 0.6877 - val_loss: 0.8001 - val_acc: 0.7220
Epoch 5/10
50000/50000 [==============================] - 138s 3ms/sample - loss: 0.8690 - acc: 0.6991 - val_loss: 0.7540 - val_acc: 0.7324
Epoch 6/10
50000/50000 [==============================] - 138s 3ms/sample - loss: 0.8395 - acc: 0.7088 - val_loss: 0.8055 - val_acc: 0.7217
Epoch 7/10
50000/50000 [==============================] - 139s 3ms/sample - loss: 0.8220 - acc: 0.7140 - val_l

## Evaluate model

In [28]:
scores = model.evaluate(xtest, ytest, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 73.72%
